In [77]:
from pathlib import Path
import sys
import sqlite3
import pandas as pd

sys.path.insert(0, str(Path.cwd().parent))
from rd.mol import smiles_to_xyz_block
from util.py3 import from_xyz_block
from util.sql import execute_append
from IPython.display import clear_output

db = Path.home() / "C5O-Kinetics/db/data.db"
conn = sqlite3.connect(db)

smiles_text = str(input("Enter a SMILES string: "))

df_smiles = pd.read_sql_query(
    "SELECT * FROM smiles WHERE smiles_text = ?", conn, params=(smiles_text,)
)

if df_smiles.empty:
    while True:
        clear_output()
        multiplicity = str(input("Indicate the multiplicity: "))
        initial_xyz = smiles_to_xyz_block(smiles_text)
        from_xyz_block(initial_xyz)

        (proceed := True) if (
            input(
                f"Type (Y)es or (N)o to continue with shown structure & multiplicity {multiplicity}."
            ).lower()[0]
            == "y"
        ) else (proceed := False)

        if proceed:
            break

    smiles_id = execute_append(
        "INSERT INTO smiles (smiles_text, multiplicity, initial) VALUES (?, ?, ?)",
        (smiles_text, multiplicity, initial_xyz),
        db,
        return_id=True,
    )
else:
    smiles_id = df_smiles["smiles_id"].iloc[0]

df_methods = pd.read_sql_query("SELECT * FROM methods", conn)
df_calculations = pd.read_sql_query(
    f"SELECT * FROM calculations WHERE smiles_id = {smiles_id}", conn
)

clear_output()
goat = True if df_calculations.empty else False
if goat:
    mask = df_methods.method == "GOAT"
    print("\n  Select an Available Method For Calculation")
else:
    mask = df_methods.method_id.isin(df_calculations.method_id)
    print("\n  Select A Method Output To Pull Starting Structure")

methods_dict = {
    row.method_id: f"  {row.method_id:<10} | {row.method:<15} | {row.functional} {row.basis}"
    for row in df_methods[mask].itertuples()
}

print("  " + "=" * 55)
for val in methods_dict.values():
    print(val)

while True:
    sel_id = int(input("\nSelect a method id: "))
    if sel_id in list(methods_dict.keys()):
        break

if not goat:
    clear_output()
    methods_dict = {
        row.calc_id: f"  {row.calc_id:<10} | {row.scan_idx1:<2} to {row.scan_idx2:<2}"
        for row in df_calculations[df_calculations.method_id == sel_id].itertuples()
    }
    if len(methods_dict) > 1:
        print("\n  Multiple Matching Calculations. Select A Calculation To Use As Starting Structure")
        print("  " + "=" * 55)
        for val in methods_dict.values():
            print(val)
        while True:
            calc_id = int(input("\nSelect a calculation id: "))
            if calc_id in list(methods_dict.keys()):
                break
    else:
        calc_id = list(methods_dict.keys())[0]

print([row for row in df_calculations[df_calculations.calc_id == calc_id].itertuples()])

ValueError: molecule has no atoms